In [15]:
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Wed, 09 Oct 2019 10:28:12 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '14973', 'Connection': 'keep-alive', 'Server': 'mw1249.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.22-1+0~20190902.26+debian9~1.gbpd64eb7+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Mon, 23 Sep 2019 14:55:44 GMT', 'Backend-Timing': 'D=101517 t=1570460144346446', 'Content-Encoding': 'gzip', 'X-Varnish': '225776502 251801884, 913721586 132514133, 294316039 619414934', 'Age': '156746', 'X-Cache': 'cp1081 hit/7, cp3030 hit/3, cp3041 hit/13', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=09-Oct-2019;Path=/;Http

In [17]:
web_soup = BeautifulSoup(result.content, 'html.parser')
table_str = web_soup.find('table')
alltrs = table_str.find_all('tr')
rows = []
for tr in alltrs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
Rec = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        Rec.append([postalcode, borough, neighborhood])

# Rec

In [18]:
cols = ['PostalCode', 'Borough', 'Neighborhood'] #The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(Rec, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(211, 3)


In [19]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page,
# you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
# These two rows will be combined into one row with the neighborhoods separated with a comma
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [20]:
df.loc[df['PostalCode'] == 'M5A']


,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [21]:

df.shape

(103, 3)

In [22]:
dfgeo = pd.read_csv("C:/Users/P7576/Desktop/Learning on Cousera/Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [23]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')


In [24]:
df2.loc[df2['PostalCode'] == 'M5G']


,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [25]:

df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [26]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [28]:
CLIENT_ID = 'GSKMHCVTNZKPLYCE3KQMFY5TOKALWAEODC4IQCNPQW0RJKNT' # your Foursquare ID
CLIENT_SECRET = 'KSLZQ2INRQFXSSAEAFV34VKAXHPHA3SPLBOYL5RDTQRAQD5C' # your Foursquare Secret
VERSION = '20180605'

In [29]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [30]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GSKMHCVTNZKPLYCE3KQMFY5TOKALWAEODC4IQCNPQW0RJKNT&client_secret=KSLZQ2INRQFXSSAEAFV34VKAXHPHA3SPLBOYL5RDTQRAQD5C&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'